In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
from selenium import webdriver
from bs4 import BeautifulSoup
from multiprocessing import Pool, Value, freeze_support

In [2]:
daum_data = pd.read_csv('금융빅데이터공모전_데이터/다음소프트_원문정보.csv', encoding='cp949')
daum_data.head()

,키워드,일자,채널,제목,작성자(언론사명),작성일자,URL
0,종로,20190630,ko.news,"[LCK 서머] '바이퍼' 박도현 ""리라, 어떤 게임 될지 기대되고 설레""",포모스,2.019060e+13,https://sports.news.naver.com/esports/news/rea...
1,종로,20190630,ko.news,"[롤챔스] '압도적인 힘' 그리핀, 킹존 격파하고 선두 재탈환(종합)",OSEN,2.019060e+13,https://sports.news.naver.com/esports/news/rea...
2,종로,20190630,ko.news,다시 1위 도약한 그리핀...2019 LCK 서머 순위현황(6월 30일),포모스,2.019060e+13,https://sports.news.naver.com/esports/news/rea...
3,종로,20190630,ko.news,"[LCK 서머] 운영-전투 균형 이룬 그리핀, 킹존에 2대 0완승(종합)",포모스,2.019060e+13,https://sports.news.naver.com/esports/news/rea...
4,종로,20190630,ko.news,"[롤챔스] 킹존-그리핀, LCK 서머 스플릿 15번째 매진",OSEN,2.019060e+13,https://sports.news.naver.com/esports/news/rea...


In [ ]:
for col in daum_data.columns:
    print('------------', col, '|------------')
    print(daum_data[col].value_counts())
    print()

In [3]:
daum_data[daum_data.채널 == 'ko.insta'].키워드.value_counts()

종로      45888
을지로     30993
중구      28582
익선동     27025
서촌      12522
힙지로      6324
중리단길       26
Name: 키워드, dtype: int64

In [ ]:
f = open('insta_crawling_서촌.txt', 'r', encoding='utf-8')
len(f.readlines())

In [ ]:
# twitter URL array

url_arr = daum_data[daum_data.채널 == 'ko.twitter'].URL.values
url_arr[10]

In [ ]:
# twitter text crawling test

if True:
    keyword = '익선동'
    keyword_url_arr = daum_data[(daum_data.채널 == 'ko.twitter') & (daum_data.키워드 == keyword)].URL.values
    
    f = open('tweets_crawling_{}.txt'.format(keyword), 'w', encoding='utf-8')

    for url in keyword_url_arr:
        try:
            raw = requests.get(url, headers = {'User-agent' : 'Mozilla/5.0'})
            html = BeautifulSoup(raw.text, 'html.parser')
            container = html.select_one('div.permalink-inner.permalink-tweet-container > div > div.js-tweet-text-container > p')
            text = container.text
            text = re.sub("[\n\u200b]", " ", text)  # \n, \u 공백으로 변경 
            text = re.sub("\s+", " ", text)  # 여러 공백 하나로 통일
            text = re.sub("[http://]\w*","", text)  # 주소 제거
            text = re.sub("[_]?[pic.twitter.com]\w*", "", text)  # twitter 주소 제거
            # print(text)
            try:
                f.write(text+'\n')
            except:
                pass
        except:
            pass
    
    f.close()

In [5]:
for k in np.unique(daum_data.키워드.values):
    print(k)

서촌
을지로
익선동
종로
중구
중리단길
힙지로


In [ ]:
# twitter text crawling

for keyword in np.unique(daum_data.키워드.values):
    keyword_url_arr = daum_data[(daum_data.채널 == 'ko.twitter') & (daum_data.키워드 == keyword)].URL.values
    
    f = open('tweets_crawling_{}.txt'.format(keyword), 'w', encoding='utf-8')
    
    print('------------------{}-----------------'.format(keyword))
    
    for url in keyword_url_arr:
        try:
            raw = requests.get(url, headers = {'User-agent' : 'Mozilla/5.0'})
            html = BeautifulSoup(raw.text, 'html.parser')
            container = html.select_one('div.permalink-inner.permalink-tweet-container > div > div.js-tweet-text-container > p')
            text = container.text
            text = re.sub("[\n\u200b]", " ", text)  # \n, \u 공백으로 변경
            text = re.sub("\s+", " ", text)  # 여러 공백 하나로 통일
            text = re.sub("[http://]\w*","", text)  # 주소 제거
            text = re.sub("[_]?[pic.twitter.com]\w*", "", text)  # twitter 주소 제거
            try:
                f.write(text+'\n')
            except:
                pass
        except:
            pass
    
    f.close()

In [6]:
# selenium 속도 향상용

options = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'cookies': 2, 'images': 2, 'javascript': 2, 
                            'plugins': 2, 'popups': 2, 'geolocation': 2, 
                            'notifications': 2, 'auto_select_certificate': 2, 'fullscreen': 2, 
                            'mouselock': 2, 'mixed_script': 2, 'media_stream': 2, 
                            'media_stream_mic': 2, 'media_stream_camera': 2, 'protocol_handlers': 2, 
                            'ppapi_broker': 2, 'automatic_downloads': 2, 'midi_sysex': 2, 
                            'push_messaging': 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2, 
                            'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement': 2, 
                            'durable_storage': 2}}
options.add_experimental_option('prefs', prefs)
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
# 크롬창을 띄우지 않는 옵션을 넣는다	
options.add_argument('headless')	
options.add_argument('disable-gpu')	

In [7]:
# instagram text crawling code

for k in np.unique(daum_data.키워드.values)[6:]:
    keyword = k
    number = 0
    keyword_url_arr = daum_data[(daum_data.채널 == 'ko.insta') & (
        daum_data.키워드 == keyword)].URL.values
   
    f = open('insta_crawling_{}.txt'.format(keyword), 'w', encoding='utf-8')
    
    print('------------------{}-----------------'.format(keyword))
    
    for url in keyword_url_arr:
        try:
            driver = webdriver.Chrome('./chromedriver', options=options)
            driver.get(url)

            taglist = driver.find_elements_by_xpath(
                "//meta[@property='instapp:hashtags']")

            text = ' '.join([i.get_attribute("content") for i in taglist])
            # print(text)
            try:
                if text != '':  # 데이터가 있으면 파일에 추가
                    f.write(text+'\n')
                    number += 1
                    if number >= 8000:
                        break
            except:
                pass
        except:
            pass
    f.close()

------------------힙지로-----------------


In [ ]:
# multiprocessing
# instagram text crawling code
# 실행 안 됨 => 어떻게 수정해야하지..

global f

def crawling(url):
    try:
        driver = webdriver.Chrome('./chromedriver', options=options)
        driver.get(url)

        taglist = driver.find_elements_by_xpath(
                "//meta[@property='instapp:hashtags']")

        tag_list = []
        tag_list = [i.get_attribute("content") for i in taglist]
        text = ' '.join(tag_list)
        print(tag_list)
        print(text)
        try:
            if tag_list != [] and text != '':  # 데이터가 있으면 파일에 추가
                f.write(text+'\n')
        except:
            pass
    except:
        pass

for k in np.unique(daum_data.키워드.values)[0:1]:
    keyword = k
    keyword_url_arr = daum_data[(daum_data.채널 == 'ko.insta') & (
        daum_data.키워드 == keyword)].URL.values
    
    f = open('insta_crawling_{}.txt'.format(keyword), 'w', encoding='utf-8')
    
    print('------------------{}-----------------'.format(keyword))
    
    p = Pool(4)
    p.map(crawling, keyword_url_arr[:10])
    p.close()
       
    f.close()

In [ ]:
# 4 core 실행

p = Pool(4)
p.map(crawling, keyword_url_arr)
p.close()

In [ ]:
keyword = '익선동'
keyword_url_arr = daum_data[(daum_data.채널 == 'ko.insta') & (daum_data.키워드 == keyword)].URL.values

url = keyword_url_arr[3]

crawling(keyword_url_arr[3])